In [ ]:
import sys

package_path = '..'
if package_path not in sys.path:
    sys.path.append(package_path)

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from src.config.config import load_config
from src.processor.loader import HDF5FileLoader
from src.filesystem import get_run_scan_directory

In [ ]:
config = load_config()
load_dir = config.path.load_dir
file = get_run_scan_directory(load_dir, 144, 1, 1)
loader = HDF5FileLoader(file)
data = loader.get_data()

poff_images = data["poff"]
poff_qbpm = data["poff_qbpm"]
print(f"{poff_images.shape=}")
print(f"{poff_qbpm.shape=}")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.imshow(np.log1p(poff_images.sum(0)))
ax.set_title("Pump Off Images Sum")

In [ ]:
from roi_rectangle import RoiRectangle

roi_rect = RoiRectangle(380, 300, 480, 380)
poff_roi_images = roi_rect.slice(poff_images)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.imshow(np.log1p(poff_roi_images.sum(0)))
ax.set_title(f"Pump Off ROI Images Sum ({roi_rect.to_tuple()})")

In [ ]:
intensities = poff_roi_images.sum(axis=(1, 2))
delays = loader.delay
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.scatter(poff_qbpm, intensities)
ax.set_title(f"Pump Off QBPM-Intensity")

In [ ]:
image = poff_roi_images[0]

hist, bins = np.histogram(image.flatten(), bins=500, range=[-1,30])

# 히스토그램 그리기
plt.figure(figsize=(10, 6))
plt.hist(image.flatten(), bins=500, range=[-1,30], color='gray', alpha=0.7)
plt.title('Image Histogram')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.ylim(None, 800)
plt.grid(True)
plt.show()

In [ ]:
image = np.maximum(poff_roi_images[0], 0)

hist, bins = np.histogram(image.flatten(), bins=500, range=[-1,30])

# 히스토그램 그리기
plt.figure(figsize=(10, 6))
plt.hist(image.flatten(), bins=500, range=[-1,30], color='gray', alpha=0.7)
plt.title('Image Histogram')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.ylim(None, 800)
plt.grid(True)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.imshow(np.log1p(np.maximum(poff_roi_images, 0).sum(0)))
ax.set_title(f"Pump Off ROI Images Sum ({roi_rect.to_tuple()})")

In [ ]:
from src.preprocessor.generic_preprocessors import subtract_dark
no_zeros = np.maximum(poff_images, 0)
no_zero_sub_darks = subtract_dark(no_zeros)

roi_no_zero_sub_darks = roi_rect.slice(no_zero_sub_darks)
fig, ax = plt.subplots(2, 1, figsize=(10, 8))
ax.imshow(np.log1p(roi_no_zero_sub_darks.sum(0)))
ax.set_title(f"Pump Off Roi Images Sum sub Dark")

In [ ]:
from src.preprocessor.image_qbpm_preprocessor import remove_outliers_using_ransac
ransac_poff_images, ransac_poff_qbpm = remove_outliers_using_ransac((no_zero_sub_darks, poff_qbpm))
roi_ransac_poff_images = roi_rect.slice(ransac_poff_images)
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.imshow(np.log1p(roi_ransac_poff_images.sum(0)))

In [ ]:
image = roi_ransac_poff_images[0]
hist, bins = np.histogram(image.flatten(), bins=500, range=[-1,30])

# 히스토그램 그리기
plt.figure(figsize=(10, 6))
plt.hist(image.flatten(), bins=500, range=[-1,30], color='gray', alpha=0.7)
plt.title('Image Histogram')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.ylim(None, 800)
plt.grid(True)
plt.show()